In [2]:
# Empezamos importando las librerías necesarias.
import itertools
import cv2
import numpy as np
from imutils.feature import FeatureDetector_create, \
    DescriptorExtractor_create, DescriptorMatcher_create

# Como parámetros de entrada podemos especificar:
#  1. Las imágenes a ser comparadas.
#  2. El detector de puntos clave a usar.
#  3. El descriptor de features que vamos a usar.
#  4. La función de similitud o "feature matcher" que aplicaremos.
#  5. Un flag para indicar si queremos visualizar los resultados o no.


# detectores de keypoint
all_keypoints = ["DENSE", "FAST", "GFTT"]
#["BRISK", "DENSE", "FAST", "GFTT", "HARRIS", "MSER", "ORB"]

# all_keypoints = ["DOG","SIFT","SURF",]


all_descriptors = ["ORB"]


all_matchers = ["BruteForce", "BruteForce-Hamming", "FlannBased"]
# all_matchers = ["FlannBased"]


for kp, desc in itertools.product(all_keypoints, all_descriptors):

    print('-------', kp, '--', desc, '--------')
    # instanciamos el detector de puntos claves
    detector = FeatureDetector_create(kp)

    # Instanciamos el extractor/descriptor
    extractor = DescriptorExtractor_create(desc)

    # Instanciamos el "feature matcher"

    # matcher = DescriptorMatcher_create(match)
    matcher = cv2.BFMatcher()

    # Cargamos en memoria las imágenes a ser comparadas.
    image1 = cv2.imread('1173878dup/18814714.jpg')
    # image2 = cv2.imread('1173878dup/18814718.jpg')
    image2 = cv2.imread('1173903dup/18815391.jpg')

    # Luego, convertimos ambas imágenes a escala de grises.
    first_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    second_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Calculamos los puntos clave de las dos imágenes.
    first_keypoints = detector.detect(first_gray)
    second_keypoints = detector.detect(second_gray)

    # Calculamos los vectores descriptivos de ambas imágenes
    first_keypoints, first_features = extractor.compute(first_gray,
                                                        first_keypoints)
    second_keypoints, second_features = extractor.compute(second_gray,
                                                          second_keypoints)

    # Utilizamos KNN para llevar a cabo la vinculación de los
    # vectores de la primera imagen con los de la segunda. El último
    # parámetro indica que queremos los dos vectores más cercanos
    # por cada par.
    raw_matches = matcher.knnMatch(first_features, second_features, 2)
    matches = []

    for match in raw_matches:
        # Tenemos que verificar que cada match sea válido. Para
        # ello, un match debe tener exactamente dos elementos
        # para poder aplicar el test de Lowe, el cual rechaza
        # aquellos matches donde la distancia entre los dos
        # mejores vectores (match[0] y match[1]) esté por encima
        # de 0.8.
        if (len(match) == 2 and
                match[0].distance < match[1].distance * 0.55):
            # Si pasamos la prueba, nos quedamos con el mejor
            # match, match[0]
            matches.append((match[0].trainIdx,
                            match[0].queryIdx))
    # Imprimimos el número de puntos clave de la imagen 1, la
    # imagen 2, así como el número de matches entre ambas.
    print(f'Número de puntos clave en la primera imagen: '
          f'{len(first_keypoints)}')
    print(f'Número de puntos clave en la segunda imagen: '
          f'{len(second_keypoints)}')
    print(f'Número de matches en total: {len(raw_matches)}')
    print(f'Número de matches: {len(matches)}')

    # Ahora es momento de visualizar. Empezamos extrayendo las
    # dimensiones de ambas imágenes.
    first_height, first_width = image1.shape[:2]
    second_height, second_width = image2.shape[:2]
    # Creamos un lienzo sobre el que dibujaremos ambas imágenes
    # junto con líneas que unan los matches entre ellas.
    visualization = np.zeros((max(first_height, second_height),
                              first_width + second_width, 3),
                             dtype='uint8')
    # Añadimos la primera imagen al lado izquierdo de la
    # visualización.
    visualization[:first_height, :first_width] = image1
    # Añadimos la segunda imagen al lado derecho de la
    # visualización.
    visualization[:second_height, first_width:] = image2
    # El ciclo de abajo lo que hace es iterar por cada match,
    # dibujando una línea que parta de la primera imagen,
    # unbicada a la izquierda, y que termine en la segunda,
    # localizada en la parte derecha.
    for train_index, query_index in matches:
        # Escogemos un color aleatorio para la línea.
        color = np.random.randint(0, high=255, size=(3,))
        color = tuple(map(int, color))
        # Calculamos el primer extremo de la línea.
        first_point = (int(first_keypoints[query_index].pt[0]),
                       int(first_keypoints[query_index].pt[1]))
        # Calculamos el segundo extremo de la línea.
        second_point = (int(second_keypoints[train_index].pt[0]
                            + first_width),
                        int(second_keypoints[train_index].pt[1]))
        # Dibujamos la línea.
        cv2.line(visualization, first_point, second_point,
                 color, 2)
    cv2.imshow('Matched', visualization)
    cv2.waitKey(0)
    print('________________________________________')


------- DENSE -- ORB --------
Número de puntos clave en la primera imagen: 2701
Número de puntos clave en la segunda imagen: 3796
Número de matches en total: 2701
Número de matches: 10
________________________________________
------- FAST -- ORB --------
Número de puntos clave en la primera imagen: 2459
Número de puntos clave en la segunda imagen: 1976
Número de matches en total: 2459
Número de matches: 2
________________________________________
------- GFTT -- ORB --------
Número de puntos clave en la primera imagen: 1471
Número de puntos clave en la segunda imagen: 967
Número de matches en total: 1471
Número de matches: 0
________________________________________
